In [1]:
from metrics_core.audio import Audio
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('./data/dataset_files/crowd-source__asr_2021-11-08__final__ALL.csv')
dnndf = pd.read_csv('./data/dataset_files/DNN.csv')
crowd_df = df[df['steps.stepCategory']=='sentence']
dnncrowd = dnndf[dnndf['sample'].isin(['crowd', 'crowd_issue'])]
dnnids = list(dnncrowd['File_id'])
crowd_df = crowd_df[crowd_df['steps.participantId'].isin(dnnids)]


/home/prad.kadambi/anaconda3/envs/pykaldi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5,50,51,52,53,54,55,56,57,58,59,60,62,63,64,65,66,67,68,69,73,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,150,151,152,153,154,155,156,157,169,170,171,172,173,174,335,336,338,339,340,341,342,343,344,345,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,371,372,373,374,375,376,377,378,379,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,450,451,452,453,454,455,456,457,458,459,460,461,462,463,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,486,579) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
def s3_to_npscale(bucket, obj_key):
    data = Audio.from_wav_s3_bucket(bucket=bucket, object_key=obj_key)
    return data.audio_signal / (2 ** 15)

In [4]:
crowd_df.columns[:50]

Index(['session.appDevice', 'session.appName', 'session.appPlatform',
       'session.appVersion', 'session.createdTime', 'session.inClinic',
       'session.language', 'session.microphone', 'session.participantId',
       'session.partitionId', 'session.sessionCount', 'session.sessionId',
       'session.sessionScriptId', 'session.sortId', 'session.startTime',
       'session.totalSteps', 'session.type', 'session.uuid',
       'steps.partitionId', 'steps.sortId', 'steps.type',
       'steps.participantId', 'steps.sessionId', 'steps.stepId',
       'steps.createdTime', 'steps.s3Bucket', 'steps.s3Path',
       'steps.elicitType', 'steps.microphone', 'steps.stepCategory',
       'steps.startTime', 'steps.endTime', 'steps.status', 'steps.name',
       'steps.sequenceNumber', 'steps.rating', 'steps.singleSelect',
       'steps.uuid', 'steps.promptListSize', 'steps.promptType',
       'steps.prompt', 'steps.originalSampleRate', 'steps.stepData',
       'transcript.simple.human', 'transcript

In [5]:
stepid_to_transcript = \
{37: 'The supermarket chain shut down because of poor management', 
38: 'Much more money must be donated to make this department succeed', 
39: 'In this famous coffee shop they serve the best donuts in town',
40: 'The chairman decided to pave over the shopping center garden', 
41: 'The standards committee met this afternoon in an open meeting'}

In [6]:
dnncrowd.shape

(231, 43)

In [7]:
'''just checking that theres only one session per participant id'''
correct =  0
notfounds = 0
nmultimplesess = 0
# participant_ids = crowd_df['session.participantId']
participant_ids = list(dnncrowd['File_id'])
df_partids = np.array(list(crowd_df['session.participantId']))
for pid in participant_ids:
    inds = np.argwhere(df_partids == pid).ravel()
    sids = np.unique(crowd_df.iloc[inds]['session.sessionId'])
    if len(sids)>1:
        print(sess)
        nmultimplesess += 1
    elif len(sids)==1:
        # print('correct')
        correct += 1
    else:
        # print('not found')
        notfounds+= 1
print('num multiple sess:\t', nmultimplesess)
print('corrects:\t', correct)
print('notfounds:\t', notfounds)

num multiple sess:	 0
corrects:	 231
notfounds:	 0


In [8]:
pid = df_partids[0]
inds = np.argwhere(df_partids==pid).ravel()
sess = crowd_df.iloc[inds]['session.sessionId'].iloc[0]
print(sess)
# print(pid)

1615491157612.0


In [9]:
sess in list(crowd_df['session.participantId'])

False

In [10]:
from gop_helper_fns import *

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/prad.kadambi/anaconda3/envs/pykaldi/lib/python3.7/site-packages/transformers/configuration_utils.py:354: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [11]:
from src.Charsiu import charsiu_forced_aligner
from alignment_helper_fns import *
charsiu = charsiu_forced_aligner('charsiu/en_w2v2_fc_10ms')
alignment_dir = './crowd_alignments/'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
n_nan = 0
for pid in tqdm.tqdm(participant_ids):
    session_artp_validation_dict[pid] = {}
    inds = np.argwhere(df_partids==pid).ravel()
    _sessid = crowd_df.iloc[inds]['session.sessionId'].iloc[0]
    _sessidz = int(_sessid)
    for step in steps:     
        _transcript = stepid_to_transcript[step]
        _step_audiopath = '/'.join([pid, '%d/%d.wav' % (_sessid, step)])
        # audio = Audio.from_wav_s3_bucket(bucket = BUCKET,
                                         # object_key = _step_audiopath)
        try:
            audio_signal = s3_to_npscale(BUCKET, _step_audiopath)
        except:
            n_nan+=1
# print(n_nan)

  0%|                                                   | 0/231 [00:00<?, ?it/s]


NameError: name 'session_artp_validation_dict' is not defined

In [12]:
participantids = dnncrowd['File_id']
BUCKET = 'a2services-prod-sessionartifacts-crowdsource'
df_partids = np.array(list(crowd_df['session.participantId']))
artps_array = np.zeros([len(df_partids), 5])
steps = stepid_to_transcript.keys()
results_dict = './crowd_validation_artps_fixed.pkl'
if not os.path.exists(results_dict):
    session_artp_validation_dict = {}
    for ii, pid in tqdm.tqdm(enumerate(participantids)):
        session_artp_validation_dict[pid] = {}
        inds = np.argwhere(df_partids==pid).ravel()
        _sessid = crowd_df.iloc[inds]['session.sessionId'].iloc[0]
        _sessidz = int(_sessid)
        for jj, step in enumerate(steps):     
            _transcript = stepid_to_transcript[step]
            _step_audiopath = '/'.join([pid, '%d/%d.wav' % (_sessid, step)])
            # audio = Audio.from_wav_s3_bucket(bucket = BUCKET,
                                             # object_key = _step_audiopath)
            try:
                audio_signal = s3_to_npscale(BUCKET, _step_audiopath)
                # len(audio_signal)
                # break
                #do alignment
                # output_tg_dir = os.path.join(alignment_dir, pid, _sessid + '-' + str(step)+'.TextGrid')
                # charsiu.serve()
                #calculate artp
                mean_gop, phonewise_gop = calculate_GOP_e2e(audio=audio_signal, transcript=_transcript)
                session_artp_validation_dict[pid][step] = mean_gop
                artps_array[ii, jj] = mean_gop
                # print(mean_gop)
            except:
                session_artp_validation_dict[pid][step] = np.nan
                artps_array[ii, jj] = np.nan
                
            # break
        # break


231it [19:42,  5.12s/it]


ModuleNotFoundError: No module named 'pkl'

In [17]:
import pickle as pkl
pkl.dump(session_artp_validation_dict, open(results_dict, 'wb'))

In [13]:
crowd_df['steps.participantId'][0]
# crowd_df['steps.partitionId'][0]

'e81acb2d-4cf9-47a8-a12a-f2ef030970a0'

In [14]:
# dnndf['']
bucket = crowd_df['steps.s3Bucket'][0]
sessid = crowd_df['steps.sessionId'][0]
objkey = crowd_df['steps.participantId'][0]

In [ ]:
Audio.from_wav_s3_bucket(bucket=)